# Avance 3 – Data Warehouse + ETL en Snowflake
Proyecto Integrador – Fleetlogix

Este notebook implementa el Data Warehouse analítico de FleetLogix en Snowflake, junto con un pipeline ETL que extrae información desde PostgreSQL, calcula métricas clave y carga datos históricos y de negocio.

## Tecnologías Utilizadas
- Snowflake (Data Warehouse)
- PostgreSQL (fuente transaccional)
- Python (pandas, psycopg2, snowflake-connector)
- schedule (automatización)

## Estructura del Proyecto
avance3/
│── etl_pipeline.py
│── ddl_snowflake.sql
│── daily_totals.sql
└── diagramas/

## Contenido del Modelo Estrella
Tabla de Hechos
- fact_deliveries

Dimensiones
- dim_date
- dim_time
- dim_driver (SCD Type 2)
- dim_vehicle (SCD Type 2)
- dim_customer
- dim_route

## Ejemplo Snowflake
CREATE TABLE fact_deliveries (
    delivery_id INT,
    driver_id INT,
    vehicle_id INT,
    route_id INT,
    delivery_date DATE,
    delivery_time TIME,
    fuel_used DECIMAL(10,2),
    total_cost DECIMAL(10,2),
    status VARCHAR(20)
);

## Pipeline ETL
Extracción: consulta datos del día anterior desde PostgreSQL  
Transformación: calcula tiempos, retrasos, eficiencia, ingresos, costos, entregas/hora  
Carga: inserta dimensiones (con SCD), tabla de hechos y tabla daily_totals  
Automatización: programado para las 02:00 AM mediante schedule

## Ejemplo de fragmento:

In [ ]:
import psycopg2
import pandas as pd
from snowflake.connector import connect

# Conexión a PostgreSQL
pg_conn = psycopg2.connect("dbname=fleetlogix user=postgres password=***")

# Extracción
df = pd.read_sql("SELECT * FROM deliveries WHERE scheduled_datetime >= CURRENT_DATE - INTERVAL '1 day'", pg_conn)

# Transformación
df['delay_minutes'] = (df['delivered_datetime'] - df['scheduled_datetime']).dt.total_seconds() / 60

# Carga en Snowflake
sf_conn = connect(user='...', password='...', account='...')
cursor = sf_conn.cursor()
for _, row in df.iterrows():
    cursor.execute("INSERT INTO fact_deliveries VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)", tuple(row))


## Cómo Ejecutar el ETL
1. Configurar variables de conexión:
   - PostgreSQL
   - Snowflake
2. Instalar dependencias:
   ```bash
   pip install psycopg2 pandas snowflake-connector-python schedule
